In [14]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [15]:
import sys
sys.path.append('../../../')

from tensorflow_models import DNNClassifier, CNNClassifier, GridSearchCNN, GridSearchDNN, RandomSearchCNN
import tensorflow as tf

In [27]:
processedDataX_path = "../../../preprocessedSamples_cut_X_samples_allGuitar_20480_Mm7_R1D.data"
processedDatay_path = "../../../preprocessedSamples_cut_y_samples_allGuitar_20480_Mm7_R1D.data"
processedData_path = ""

if os.path.isfile(processedDataX_path): #if already preprocessed
    processedX = np.load(processedDataX_path)
    processedy = np.load(processedDatay_path)
else:
    import pandas as pd
    import librosa
    df = pd.read_csv('../../samples_allGuitar_20480_Mm7_R1D.csv')

    X_load = np.array(df.iloc[:,:-1], dtype=np.float)
    y_load = np.array(df.iloc[:,-1], dtype=np.float)
    processedX = np.zeros((len(X_load),12,80,1), dtype=np.float)
    processedy = np.zeros(len(y_load), dtype=np.float)
    X_fft_new = np.zeros(20480)
    for i in range(len(X_load)):
        sample = np.fft.rfft(X_load[i])
        for ii in range(len(sample)):
            if ii < 1001: #ignore frequencies greater than 2kHz
                X_fft_new[ii] = sample[ii]
                
        sample = np.fft.ifft(X_fft_new)
        sample = librosa.feature.chroma_stft(y=sample, sr=44100, n_fft=20480, hop_length=258)
        sample = np.atleast_3d(sample)
        processedX[i] = sample
        processedy[i] = y_load[i]
        if i % 400 == 0:
            print(i)

    processedX.dump(processedDataX_path)
    processedy.dump(processedDatay_path)

from sklearn.utils import shuffle
print(processedy)
sprocessedX, sprocessedy = shuffle(processedX, processedy)
print(len(sprocessedX))

trainRange = int(len(sprocessedX) * 0.8)
validRange = int(len(sprocessedX) * 9)
testRange = int(len(sprocessedX) * 0.1)


X_train = np.array(sprocessedX[:trainRange], dtype=np.float)
y_train = np.array(sprocessedy[:trainRange], dtype=np.float)

X_valid = np.array(sprocessedX[trainRange:validRange], dtype=np.float)
y_valid = np.array(sprocessedy[trainRange:validRange], dtype=np.float)

X_test = np.array(sprocessedX[testRange:], dtype=np.float)
y_test = np.array(sprocessedy[testRange:], dtype=np.float)
print(y_test[1])
print(sprocessedX.shape)

C:\Users\Denis\AppData\Local\conda\conda\envs\tensorflow_env_gpu\lib\site-packages\ipykernel_launcher.py:22: ComplexWarning: Casting complex values to real discards the imaginary part


0
400
800
1200
1600
2000
2400
2800
3200
[  0.   0.   0. ...,  45.  45.  45.]
3214
1.0
(3214, 12, 80, 1)


In [28]:
def leaky_relu(alpha=0.01):
		def parametrized_leaky_relu(z, name=None):
			return tf.maximum(alpha * z, z, name=name)
		return parametrized_leaky_relu

cnn = CNNClassifier(batch_size=400, learning_rate=0.05, n_hidden_layers=2, n_neurons=500, 
                    optimizer_class=tf.train.AdamOptimizer, activation=leaky_relu(), dropout_rate = 0.1,
                    conv1={'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.3, 'conv1_activation':tf.nn.relu},
                    conv2={'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.2, 'conv2_activation':tf.nn.relu})

cnn.fit(X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid)

score = cnn.accuracy_score(X_test, y_test)
score

***********
(2571, 12, 80, 1)
(2571,)
(643, 12, 80, 1)
(643,)
Epoch 0, train accuracy: 13.5514%, valid. accuracy: 12.4417%, valid. best loss: 3.845730
Epoch 1, train accuracy: 30.3738%, valid. accuracy: 27.9938%, valid. best loss: 3.604322
Epoch 2, train accuracy: 41.3551%, valid. accuracy: 36.0809%, valid. best loss: 2.995095
Epoch 3, train accuracy: 56.5421%, valid. accuracy: 47.7449%, valid. best loss: 2.351482
Epoch 4, train accuracy: 62.6168%, valid. accuracy: 53.6547%, valid. best loss: 1.974236
Epoch 5, train accuracy: 64.2523%, valid. accuracy: 55.8320%, valid. best loss: 1.704491
Epoch 6, train accuracy: 67.7570%, valid. accuracy: 57.5428%, valid. best loss: 1.534719
Epoch 7, train accuracy: 73.8318%, valid. accuracy: 64.5412%, valid. best loss: 1.348875
Epoch 8, train accuracy: 76.8691%, valid. accuracy: 67.9627%, valid. best loss: 1.239883
Epoch 9, train accuracy: 77.1028%, valid. accuracy: 68.1182%, valid. best loss: 1.148652
Epoch 10, train accuracy: 77.8037%, valid. accur

0.95264441

In [29]:
import tensorflow as tf

def leaky_relu(alpha=0.01):
		def parametrized_leaky_relu(z, name=None):
			return tf.maximum(alpha * z, z, name=name)
		return parametrized_leaky_relu
    
"""
Best parameters (kfold = 5):

{'n_hidden_layers': 2, 'n_neurons': 500, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>, 
'learning_rate': 0.05, 'batch_size': 400, 
'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000203351C67B8>, 
'dropout_rate': 0.1, 'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 
'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x00000203335AC620>}, 'conv2': {'conv2_fmaps': 16, 
'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.2, 'conv2_activation': <function relu at 0x00000203335AC620>}, 
'architecture': 4, 'accuracy_rate': 0.9958441495895386}


Best parameters (kfold = 1 and cutting freq):

Best parameters:
{'n_hidden_layers': 8, 'n_neurons': 500, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>,
'learning_rate': 0.025, 'batch_size': 600,
'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001FDB8747950>,
'dropout_rate': 1e-06, 
'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x000001FDA77410D0>}, 
'conv2': {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.3, 'conv2_activation': <function relu at 0x000001FDA77410D0>}, 'architecture': 1, 'accuracy_rate': 0.98441553115844727}

"""

params = {'n_hidden_layers': [1,2,3,4,5,6,7,8,9,10],
	'n_neurons' : [50, 150, 250, 300, 350, 400, 450, 500],
	'optimizer_class' : [tf.train.AdamOptimizer, tf.train.AdagradOptimizer],
	'learning_rate' : [0.01, 0.05, 0.1, 0.025],
	'batch_size' : [200, 250, 300, 400, 600],
	'activation' : [leaky_relu(), tf.nn.elu],
	'dropout_rate' : [0.000001, 0.1, 0.2, 0.3, 0.4],
	'conv1' : [{'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.elu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.2, 'conv1_activation':tf.nn.relu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.3, 'conv1_activation':tf.nn.relu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.elu}],
	'conv2' : [{'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.elu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.2, 'conv2_activation':tf.nn.relu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.3, 'conv2_activation':tf.nn.relu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.elu}],
    'architecture' : [1, 2, 3, 4]
}
rnd_search = RandomSearchCNN(params, k_fold=1, num_random_combinations=300)
rnd_search.fit(X_train, y_train, X_valid, y_valid)

testing 300 combinations.
Trining CNN with parameters: n_hidden_layers: 9, n_neurons: 250, optimizer_class: <class 'tensorflow.python.training.adam.AdamOptimizer'>, learning_rate: 0, dropout_rate: 0, batch_size: 400, activation: <function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001FDB8747950>, conv1: {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x000001FDA77410D0>}, conv2: {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.1, 'conv2_activation': <function elu at 0x000001FDA77300D0>} .architecture: 2, 
shapes:
(2185, 12, 80, 1) (385, 12, 80, 1)
(2185,) (385,)
***********
(2185, 12, 80, 1)
(2185,)
(643, 12, 80, 1)
(643,)
Epoch 0, train accuracy: 4.1190%, valid. accuracy: 3.7325%, valid. best loss: 3.870077
Epoch 1, train accuracy: 3.6613%, valid. accuracy: 2.0218%, valid. best loss: 3.870077
Epoch 2, train accuracy: 3.8902%, valid. accuracy: 2.7994%, valid. best loss: 3.870077
Epo

Epoch 86, train accuracy: 17.1625%, valid. accuracy: 14.9300%, valid. best loss: 2.840019
Epoch 87, train accuracy: 18.3066%, valid. accuracy: 14.6190%, valid. best loss: 2.780087
Epoch 88, train accuracy: 18.3066%, valid. accuracy: 14.9300%, valid. best loss: 2.758785
Epoch 89, train accuracy: 16.4760%, valid. accuracy: 16.4852%, valid. best loss: 2.675735
Epoch 90, train accuracy: 17.1625%, valid. accuracy: 17.4184%, valid. best loss: 2.660971
Epoch 91, train accuracy: 16.0183%, valid. accuracy: 17.7294%, valid. best loss: 2.660971
Epoch 92, train accuracy: 14.8741%, valid. accuracy: 16.6407%, valid. best loss: 2.610215
Epoch 93, train accuracy: 17.1625%, valid. accuracy: 18.8180%, valid. best loss: 2.610215
Epoch 94, train accuracy: 21.2815%, valid. accuracy: 21.3064%, valid. best loss: 2.479588
Epoch 95, train accuracy: 24.7140%, valid. accuracy: 21.3064%, valid. best loss: 2.479588
Epoch 96, train accuracy: 15.7895%, valid. accuracy: 17.4184%, valid. best loss: 2.477934
Epoch 97, 

Epoch 178, train accuracy: 47.3684%, valid. accuracy: 44.3235%, valid. best loss: 1.624933
Epoch 179, train accuracy: 39.3593%, valid. accuracy: 41.3686%, valid. best loss: 1.618631
Epoch 180, train accuracy: 44.6224%, valid. accuracy: 38.5692%, valid. best loss: 1.613267
Epoch 181, train accuracy: 49.4279%, valid. accuracy: 44.7900%, valid. best loss: 1.613267
Epoch 182, train accuracy: 44.3936%, valid. accuracy: 41.0575%, valid. best loss: 1.580909
Epoch 183, train accuracy: 47.1396%, valid. accuracy: 44.0124%, valid. best loss: 1.580909
Epoch 184, train accuracy: 39.8169%, valid. accuracy: 40.5910%, valid. best loss: 1.580909
Epoch 185, train accuracy: 45.3089%, valid. accuracy: 41.5241%, valid. best loss: 1.580909
Epoch 186, train accuracy: 46.9108%, valid. accuracy: 43.8569%, valid. best loss: 1.580909
Epoch 187, train accuracy: 49.4279%, valid. accuracy: 41.6796%, valid. best loss: 1.580909
Epoch 188, train accuracy: 47.3684%, valid. accuracy: 44.7900%, valid. best loss: 1.580909

Epoch 270, train accuracy: 64.7597%, valid. accuracy: 56.6096%, valid. best loss: 1.325367
Epoch 271, train accuracy: 69.1075%, valid. accuracy: 59.7201%, valid. best loss: 1.321625
Epoch 272, train accuracy: 62.9291%, valid. accuracy: 56.7652%, valid. best loss: 1.319143
Epoch 273, train accuracy: 64.9886%, valid. accuracy: 58.7869%, valid. best loss: 1.319143
Epoch 274, train accuracy: 63.1579%, valid. accuracy: 60.8087%, valid. best loss: 1.319143
Epoch 275, train accuracy: 65.2174%, valid. accuracy: 57.8538%, valid. best loss: 1.317738
Epoch 276, train accuracy: 66.1327%, valid. accuracy: 59.4090%, valid. best loss: 1.317738
Epoch 277, train accuracy: 71.1670%, valid. accuracy: 60.3421%, valid. best loss: 1.317738
Epoch 278, train accuracy: 64.0732%, valid. accuracy: 58.1649%, valid. best loss: 1.260320
Epoch 279, train accuracy: 70.7094%, valid. accuracy: 59.7201%, valid. best loss: 1.260320
Epoch 280, train accuracy: 68.4211%, valid. accuracy: 60.1866%, valid. best loss: 1.260320

Epoch 46, train accuracy: 95.6044%, valid. accuracy: 82.2706%, valid. best loss: 0.706302
Epoch 47, train accuracy: 95.9707%, valid. accuracy: 82.8927%, valid. best loss: 0.706302
Epoch 48, train accuracy: 95.6044%, valid. accuracy: 81.4930%, valid. best loss: 0.706302
Epoch 49, train accuracy: 93.0403%, valid. accuracy: 81.9596%, valid. best loss: 0.706302
Epoch 50, train accuracy: 95.9707%, valid. accuracy: 81.1820%, valid. best loss: 0.706302
Epoch 51, train accuracy: 97.4359%, valid. accuracy: 81.9596%, valid. best loss: 0.706302
Epoch 52, train accuracy: 94.5055%, valid. accuracy: 81.0264%, valid. best loss: 0.706302
Epoch 53, train accuracy: 97.8022%, valid. accuracy: 83.2037%, valid. best loss: 0.706302
Epoch 54, train accuracy: 94.8718%, valid. accuracy: 81.6485%, valid. best loss: 0.706302
Epoch 55, train accuracy: 96.3370%, valid. accuracy: 83.5148%, valid. best loss: 0.706302
Epoch 56, train accuracy: 96.7033%, valid. accuracy: 81.3375%, valid. best loss: 0.706302
Epoch 57, 

...

***********
(2185, 12, 80, 1)
(2185,)
(643, 12, 80, 1)
(643,)
Epoch 0, train accuracy: 7.5549%, valid. accuracy: 4.9767%, valid. best loss: 4.090265
Epoch 1, train accuracy: 13.4615%, valid. accuracy: 11.6641%, valid. best loss: 3.636075
Epoch 2, train accuracy: 20.0549%, valid. accuracy: 18.5070%, valid. best loss: 3.403145
Epoch 3, train accuracy: 21.2912%, valid. accuracy: 21.1509%, valid. best loss: 3.253588
Epoch 4, train accuracy: 25.8242%, valid. accuracy: 23.6392%, valid. best loss: 3.052397
Epoch 5, train accuracy: 33.1044%, valid. accuracy: 28.4603%, valid. best loss: 2.753899
Epoch 6, train accuracy: 43.6813%, valid. accuracy: 37.4806%, valid. best loss: 2.424708
Epoch 7, train accuracy: 49.4506%, valid. accuracy: 42.6128%, valid. best loss: 2.118440
Epoch 8, train accuracy: 50.1374%, valid. accuracy: 44.7900%, valid. best loss: 1.941478
Epoch 9, train accuracy: 49.5879%, valid. accuracy: 46.6563%, valid. best loss: 1.786277
Epoch 10, train accuracy: 57.0055%, valid. accurac

Epoch 92, train accuracy: 92.9945%, valid. accuracy: 79.7823%, valid. best loss: 0.684030
Epoch 93, train accuracy: 93.2692%, valid. accuracy: 80.4044%, valid. best loss: 0.684030
Epoch 94, train accuracy: 92.4451%, valid. accuracy: 79.3157%, valid. best loss: 0.684030
Epoch 95, train accuracy: 91.4835%, valid. accuracy: 80.2488%, valid. best loss: 0.684030
Epoch 96, train accuracy: 93.5440%, valid. accuracy: 78.5381%, valid. best loss: 0.684030
Epoch 97, train accuracy: 92.5824%, valid. accuracy: 80.0933%, valid. best loss: 0.684030
Epoch 98, train accuracy: 94.2308%, valid. accuracy: 80.5599%, valid. best loss: 0.684030
Epoch 99, train accuracy: 92.9945%, valid. accuracy: 79.3157%, valid. best loss: 0.684030
Epoch 100, train accuracy: 92.3077%, valid. accuracy: 79.7823%, valid. best loss: 0.684030
Epoch 101, train accuracy: 92.7198%, valid. accuracy: 79.9378%, valid. best loss: 0.684030
Epoch 102, train accuracy: 93.9561%, valid. accuracy: 80.0933%, valid. best loss: 0.684030
Epoch 1

Epoch 61, train accuracy: 93.4066%, valid. accuracy: 85.2255%, valid. best loss: 0.433489
Epoch 62, train accuracy: 93.7729%, valid. accuracy: 86.1586%, valid. best loss: 0.433489
Epoch 63, train accuracy: 95.9707%, valid. accuracy: 85.2255%, valid. best loss: 0.433489
Early stopping!
Final accuracy on test set: 0.922078
******************************************
{'n_hidden_layers': 4, 'n_neurons': 150, 'optimizer_class': <class 'tensorflow.python.training.adagrad.AdagradOptimizer'>, 'learning_rate': 0.05, 'batch_size': 250, 'activation': <function elu at 0x000001FDA77300D0>, 'dropout_rate': 0.4, 'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.2, 'conv1_activation': <function relu at 0x000001FDA77410D0>}, 'conv2': {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.1, 'conv2_activation': <function elu at 0x000001FDA77300D0>}, 'architecture': 3, 'accuracy_rate': 0.92207795381546021}
******************************************
T

***********
(2185, 12, 80, 1)
(2185,)
(643, 12, 80, 1)
(643,)
Epoch 0, train accuracy: 7.0513%, valid. accuracy: 6.8429%, valid. best loss: 4.636985
Epoch 1, train accuracy: 15.7051%, valid. accuracy: 14.6190%, valid. best loss: 3.652993
Epoch 2, train accuracy: 27.2436%, valid. accuracy: 25.3499%, valid. best loss: 3.343240
Epoch 3, train accuracy: 39.7436%, valid. accuracy: 34.0591%, valid. best loss: 2.455200
Epoch 4, train accuracy: 52.5641%, valid. accuracy: 46.8118%, valid. best loss: 2.000304
Epoch 5, train accuracy: 61.8590%, valid. accuracy: 54.8989%, valid. best loss: 1.540659
Epoch 6, train accuracy: 66.6667%, valid. accuracy: 57.5428%, valid. best loss: 1.354755
Epoch 7, train accuracy: 67.6282%, valid. accuracy: 58.9425%, valid. best loss: 1.269507
Epoch 8, train accuracy: 70.1923%, valid. accuracy: 61.1197%, valid. best loss: 1.200387
Epoch 9, train accuracy: 70.8333%, valid. accuracy: 64.5412%, valid. best loss: 1.178032
Epoch 10, train accuracy: 72.4359%, valid. accurac

Epoch 10, train accuracy: 75.2860%, valid. accuracy: 66.7185%, valid. best loss: 1.110208
Epoch 11, train accuracy: 77.1167%, valid. accuracy: 64.6967%, valid. best loss: 1.084368
Epoch 12, train accuracy: 80.7780%, valid. accuracy: 68.7403%, valid. best loss: 1.080564
Epoch 13, train accuracy: 79.8627%, valid. accuracy: 69.5179%, valid. best loss: 1.012322
Epoch 14, train accuracy: 79.1762%, valid. accuracy: 70.7621%, valid. best loss: 1.012322
Epoch 15, train accuracy: 78.7185%, valid. accuracy: 70.2955%, valid. best loss: 0.997418
Epoch 16, train accuracy: 83.5240%, valid. accuracy: 71.6952%, valid. best loss: 0.951485
Epoch 17, train accuracy: 83.9817%, valid. accuracy: 72.0062%, valid. best loss: 0.941327
Epoch 18, train accuracy: 86.0412%, valid. accuracy: 73.2504%, valid. best loss: 0.941327
Epoch 19, train accuracy: 85.5835%, valid. accuracy: 73.4059%, valid. best loss: 0.904759
Epoch 20, train accuracy: 84.8970%, valid. accuracy: 71.5397%, valid. best loss: 0.903131
Epoch 21, 